[back to README](../../README.ipynb)

# Web Data Anlaysis

## Data Source 

- raw web log data available as a part of [this dataset](http://old.honeynet.org/scans/scan34/) is used for this notebook.
- the weblogs stored at s3 bucket `/essentia-playground/Weblog_Dataset/http/`

Here, I'm planning on demonstrating 
1. deviding the log data into columns (converting to table structure)
2. perform some EDAs and visualization
3. pick features for anomaly detection 
4. format the data for suitable use for Sagemaker Random Cut Forest (RCF) algorithm.

Then put the data back to different S3 bucket. Step 3 and 4 are not conclusive though, depends on the result from step 2.

## Setup
First we'll start with selecting the bucket as our datastore, then creating a category called `weblogs` and include all the raw apache web log data files in the bucket.

In [3]:
# select datastore, create category and take a peek at the data
ess select s3://essentia-playground
ess category add weblogs "/Weblog_Dataset/http/access_log.*"
ess summary weblogs

2019-11-27 22:23:32 ip-10-10-1-118 ess[3121]: Fetching file list from datastore.
2019-11-27 22:23:32 ip-10-10-1-118 ess[3121]: Examining largest matched file to determine compression type: /Weblog_Dataset/http/access_log.1
2019-11-27 22:23:32 ip-10-10-1-118 ess[3121]: Probing largest matched file to determine data configuration: /Weblog_Dataset/http/access_log.1
Name:        weblogs
Pattern:     /Weblog_Dataset/http/access_log.*
Exclude:     None
Date Format: auto
Date Regex:  
Archive:     
Delimiter:   Space
# of files:  6
Total size:  439.4KB
File range:  2019-11-01 - 2019-11-06
# columns:   10
Column Spec: IP:col_1 S:col_2 S:col_3 S:col_4 S:col_5 S:col_6 I:col_7 I:col_8 S:col_9 S:col_10
Pkey: 
Schema: IP:col_1 S:col_2 S:col_3 S:col_4 S:col_5 S:col_6 I:col_7 I:col_8 S:col_9 S:col_10
Preprocess:  
usecache:    False
Comment:    

First few lines:
24.196.254.170 - - [06/Mar/2005:05:28:52 -0500] "GET / HTTP/1.1" 403 2898 "-" "Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"
192.195.225.

## Data Cleaning

It looks like each row is an entry of server access log in [combined log format](https://httpd.apache.org/docs/2.4/logs.html#combine_log), which is one long string. <br>
This cannot be analyzed in the format, so let's break it down to detailed columns. Followings are the list of columns we can get from the file, from left to right in the row log format.

```24.196.254.170 - - [06/Mar/2005:05:28:52 -0500] "GET / HTTP/1.1" 403 2898 "-" "Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"```

* `24.196.254.170` - Client IP Address
* `-` - hyphen indicates request piece of info is not available.
* `-` - user id of the person who is requesting the documentsl
* `[06/Mar/2005:05:28:52 -0500]` - date, month, year and time of the request
* `"GET / HTTP/1.1"` - requet line given from client. This can be broken down to smaller columns
* `403` - status code that the server sent back to client. 
* `2898` - size of object returned to the cilent
* `"-"` - this can contain URLs or domain name, and is referer.
* `"Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"` - user agent http request header. 

Out of all columns, request line and time columns will be separated to more detailed columns.

We can do this using `aq_pp` command. But first, let us create a column specs. It looks like `ess category add` was unable to scan the column spec we wanted, so we can make it ourselves. 
Below is the column spec that will be used for the rest of the notebook.

```IP:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent```

**Attributes**<br>

* `sep:` attributes were used alongside wtih 
* `div` to specify separate delimiters for each columns.
* `eok` was used to skip the line which include invalid data. works as data cleaning function.

In [31]:
# Create a column spec
cols="IP:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' I:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' 'S:sign sep:'' I:diff sep:'] \"' S:request sep:'\" ' I:status sep:' ' I:size sep:' ' S:f"
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok - \
-d IP:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent | \
head -n 10

"ip","hypen","user_id","date","month","year","hour","minute","second","time_dif","request_method","requested_resource","protocol","return_code","obj_size","referrer","user_agent"
192.195.225.6,"-","-",6,"Mar",2005,5,31,37,"-0500","GET","/","HTTP/1.1",403,2898,"""-""","""Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"""
218.75.19.178,"-","-",6,"Mar",2005,6,37,20,"-0500","GET","/","HTTP/1.1",403,2898,"""-""","""Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"""
65.115.46.225,"-","-",6,"Mar",2005,7,45,21,"-0500","GET","/","HTTP/1.1",403,2898,"""-""","""Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"""
219.137.79.110,"-","-",6,"Mar",2005,9,5,38,"-0500","GET","/","HTTP/1.1",403,2898,"""-""","""Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"""
219.61.8.78,"-","-",6,"Mar",2005,9,12,31,"-0500","GET","/","HTTP/1.1",403,2898,"""-""","""Mozilla/4.0 (compatible; MSIE 5.5; Windows 98)"""
208.57.191.112,"-","-",6,"Mar",2005,10,31,42,"-0500","GET","/","HTTP/1.1",403,2898,"""-""","""Mozilla/4.0 (compat

## EDA

Now we've done the cleaning and structuring of the data, we can move onto exploratory data analysis. 

Let's start with analyzing clients' ip column.

First we'll take a look at frequency counts of each clients' IP addresses to identify where we're getting accessed from.

### Clients Ip address

We'll acheive this by using `aq_cnt`. Note that we're outputting the stream from `aq_pp` in `aq` binary internal format, so that we do not need to provide additional column spec on `aq_cnt` command. <br>
Finally the result is sorted by `aq_ord` command.

In [23]:
# stream with ess, structurize with aq_pp, then count with aq_cnt
# note that aq_pp is outputting the data in aq_tools' binary format, so that column specs do not need to be specified in aq_cnt
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d IP:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_cnt -f,aq,eok,qui - -kX,aq - key IP | \
aq_ord -f,aq - -sort,dec count 

"ip","count"
2019-12-03 18:47:13 ip-10-10-1-118 ess[20516]: !!!Warning!!! License is going to expire on 2020-01-01!
64.62.145.98,414
210.118.169.20,410
210.116.59.164,179
4.152.207.238,94
210.51.12.238,92
64.122.238.114,91
220.170.88.36,63
222.95.35.200,51
4.152.207.126,46
210.127.248.52,46
216.105.210.91,45
66.99.250.98,42
66.173.230.27,40
222.95.32.114,38
61.166.155.162,38
61.178.79.227,34
63.202.179.73,32
67.162.217.35,32
216.171.174.124,28
211.229.45.67,23
82.229.169.73,23
69.86.160.253,23
65.117.45.251,23
220.95.232.60,23
217.162.121.21,23
4.249.111.162,23
220.95.231.3,23
61.81.96.175,23
219.245.156.12,23
148.245.28.39,21
24.18.186.248,16
172.169.6.104,16
220.28.212.176,16
60.248.26.50,16
69.86.164.233,16
221.184.100.13,16
67.181.19.171,16
69.233.130.234,16
211.59.0.204,16
211.211.4.51,16
81.193.64.246,16
61.235.189.106,16
24.19.201.156,16
69.153.13.148,16
201.129.65.141,16
172.155.203.99,16
24.5.169.95,16
69.210.209.141,16
68.55.90.67,16
24.107.54.128,16
221.234.48.249,16
24.99.1

You can observe that majority of accesses are from top few addresses. We can also plot the result file (`data/ip_distr.csv`), and we'll get the below plot.

## Insert IMG HERE

It is bit hard to comprehend what is going on with all these ip addresses though. What we can do is to map these addresses into thier corresponding geolocations, so we can interpret them precisely.

**Replacing IP addresses with Regions**<br>
We've prepared a lookup table file which includes a list of IP addresses listed above and its corresponding regions and countries. We can take a look at the lookup file as well as the result file from prior cell below using `head` command.

In [3]:
lookup="data/ip_table.csv"
client_ips="data/ip_distr.csv"
head $lookup
echo 
head $client_ips

ip,count,region,country
64.62.145.98,414,California,United States
210.118.169.20,410,Gangwon-do,Korea
210.116.59.164,179,Seoul,Korea
4.152.207.238,94,New Jersey,United States
210.51.12.238,92,Beijing,China
64.122.238.114,91,Minnesota,United States
220.170.88.36,63,Hunan,China
222.95.35.200,51,Jiangsu,China
4.152.207.126,46,New Jersey,United States

"ip","count"
64.62.145.98,414
210.118.169.20,410
210.116.59.164,179
4.152.207.238,94
210.51.12.238,92
64.122.238.114,91
220.170.88.36,63
222.95.35.200,51
4.152.207.126,46


We'd like to do is the following.
1. match ip columns' values on `ip_distr.csv` file to the ip values on the lookup, and where there's a match, add country column on `ip_distr.csv` file with corresponding country value.
2. aggregate the `ip_distr.csv`'s data by same region and country, and recount the frequency counts of each. 

For the first step, we can use `-cmb` option of `aq_pp` command in order to match the records with IP columns. 

In [9]:
# matching by IP address
aq_pp -f,+1 $client_ips -d ip:ip i:count -cmb,+1,all $lookup ip,key:ip i,cmb:count S:region S:country

"ip","count","region","country"
64.62.145.98,414,"California","United States"
210.118.169.20,410,"Gangwon-do","Korea"
210.116.59.164,179,"Seoul","Korea"
4.152.207.238,94,"New Jersey","United States"
210.51.12.238,92,"Beijing","China"
64.122.238.114,91,"Minnesota","United States"
220.170.88.36,63,"Hunan","China"
222.95.35.200,51,"Jiangsu","China"
4.152.207.126,46,"New Jersey","United States"
210.127.248.52,46,"Seoul","Korea"
216.105.210.91,45,"Wisconsin","United States"
66.99.250.98,42,"Illinois","United States"
66.173.230.27,40,"Virginia","United States"
222.95.32.114,38,"Jiangsu","China"
61.166.155.162,38,"Yunnan","China"
61.178.79.227,34,"Gansu","China"
63.202.179.73,32,"New York","United States"
67.162.217.35,32,"Arkansas","United States"
216.171.174.124,28,"California","United States"
211.229.45.67,23,"Gyeonggi-do","Korea"
82.229.169.73,23,"Île-de-France","France"
69.86.160.253,23,"New York","United States"
65.117.45.251,23,"Minnesota","United States"
220.95.232.60,23,"Gyeonggi-do"

70.80.223.173,1,,
218.64.24.205,1,,
210.82.34.25,1,,
220.170.233.137,1,,
216.74.229.19,1,,
172.155.134.244,1,,
82.52.98.248,1,,
68.201.200.65,1,,
195.166.22.231,1,,
193.109.122.11,1,,
66.249.194.119,1,,
216.73.162.91,1,,
222.166.160.157,1,,
220.170.159.6,1,,
193.109.122.23,1,,
222.33.38.94,1,,
24.232.140.34,1,,
68.94.125.162,1,,
24.13.26.115,1,,
67.121.94.193,1,,
216.104.137.150,1,,
61.10.7.101,1,,
211.179.10.212,1,,
219.133.240.116,1,,
60.213.104.126,1,,
138.73.71.118,1,,
218.61.87.218,1,,
193.109.122.47,1,,
207.71.220.100,1,,
218.74.38.64,1,,
83.131.116.49,1,,
61.10.7.47,1,,
68.42.30.57,1,,
66.76.233.44,1,,
211.9.146.6,1,,
222.166.160.161,1,,
198.54.202.4,1,,
195.199.231.234,1,,
208.183.225.11,1,,
222.240.84.179,1,,
204.49.32.16,1,,
61.234.121.2,1,,
208.185.174.208,1,,
218.59.22.246,1,,
63.85.149.252,1,,
220.174.102.79,1,,
61.222.0.242,1,,
222.166.160.60,1,,
218.90.219.110,1,,
61.10.7.82,1,,
221.217.50.33,1,,
216.150.210.194,1,,
67.172.189.99,1,,
216.252.70.106,1,,
63.26.231.71,1,,
1

Step 2 uses region and country both as composite key coulmn to count the occurence of regions and countries. 


In [12]:
# count occurance of each region and country
aq_pp -f,+1 $client_ips -d ip:ip i:count -cmb,+1,all $lookup ip,key:ip i,cmb:count S:region S:country -o,aq - | \
aq_cnt -f,aq - -kX - key country

"country","count"
,338
"Bolivia",1
"United Kingdom",1
"Denmark",1
"Iran",1
"Canada",1
"Bangladesh",1
"Portugal",2
"Taiwan",5
"Japan",4
"Mexico",3
"Switzerland",2
"France",1
"China",22
"Korea",11
"United States",44


## Status Codes

Next let's look at the http status code to see how often each codes appears in the traffic. We can again use `aq_cnt` command to see the frequencies and unique values.

In [1]:
# stream with ess, structurize with aq_pp, then count with aq_cnt
# note that aq_pp is outputting the data in aq_tools' binary format, so that column specs do not need to be specified in aq_cnt
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d IP:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_cnt -f,aq,eok,qui - -kX,aq - key return_code | \
aq_ord -f,aq - -sort,dec count 

"return_code","count"
2019-12-10 20:15:43 ip-10-10-1-118 ess[2856]: !!!Warning!!! License is going to expire on 2020-01-01!
404,2228
403,599
200,140
400,76
405,59
500,46


Majority of the code is 404, which is NOT FOUND error, followed by 403 and 200. In order to visualize this result, let's convert the result into log scale using `log()` and `-eval` option in aq_pp.

In [5]:
# stream with ess, structurize with aq_pp, then count with aq_cnt
# note that aq_pp is outputting the data in aq_tools' binary format, so that column specs do not need to be specified in aq_cnt
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d IP:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_cnt -f,aq,eok,qui - -kX,aq - key return_code | \
aq_pp -f,aq,eok,qui - -eval f:log_count 'log(count)' -o,aq - | \
aq_ord -f,aq - -sort,dec count 

"return_code","count","log_count"
2019-12-10 21:01:02 ip-10-10-1-118 ess[3142]: !!!Warning!!! License is going to expire on 2020-01-01!
404,2228,7.708859601047175
403,599,6.3952615981154493
200,140,4.9416424226093039
400,76,4.3307333402863311
405,59,4.0775374439057197
500,46,3.8286413964890951


The graph looks like below.

**Log Scale Distribution Chart**
<img src='img/status_log.png'>

Majority of the codes are error codes in the data. Let's check where these requests are coming from.<br>

**source of the requests**<br>
We'll take most frequent IPs and analyze what kinds of return code each source received.

In [14]:
# stream with ess, structurize with aq_pp, then count with aq_cnt
# note that aq_pp is outputting the data in aq_tools' binary format, so that column specs do not need to be specified in aq_cnt
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d IP:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_pp -f,aq,eok,qui - -filt 'return_code != 200' -c IP request_method requested_resource return_code

"ip","request_method","requested_resource","return_code"
2019-12-10 22:21:53 ip-10-10-1-118 ess[3727]: !!!Warning!!! License is going to expire on 2020-01-01!
192.195.225.6,"GET","/",403
218.75.19.178,"GET","/",403
65.115.46.225,"GET","/",403
219.137.79.110,"GET","/",403
219.61.8.78,"GET","/",403
208.57.191.112,"GET","/",403
64.90.198.61,"GET","/",403
68.233.81.168,"GET","/scripts/root.exe?/c+dir",404
68.233.81.168,"GET","/MSADC/root.exe?/c+dir",404
68.233.81.168,"GET","/c/winnt/system32/cmd.exe?/c+dir",404
68.233.81.168,"GET","/d/winnt/system32/cmd.exe?/c+dir",404
68.233.81.168,"GET","/scripts/..%255c../winnt/system32/cmd.exe?/c+dir",404
68.233.81.168,"GET","/_vti_bin/..%255c../..%255c../..%255c../winnt/system32/cmd.exe?/c+dir",404
68.233.81.168,"GET","/_mem_bin/..%255c../..%255c../..%255c../winnt/system32/cmd.exe?/c+dir",404
68.233.81.168,"GET","/msadc/..%255c../..%255c../..%255c/..%c1%1c../..%c1%1c../..%c1%1c../winnt/system32/cmd.exe?/c+dir",404
68.233.81.168,"GET","/scripts/..%c1%1

64.62.145.98,"GET","//cgi-bin/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi-bin/",403
64.62.145.98,"GET","//cgi-bin/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi/awstats/",404
64.62.145.98,"GET","//cgi/awstats/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//.../awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi-bin/",403
64.62.145.98,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi/awstats/",404
64.62.145.98,"GET","//cgi-bin/",403
64.62.145.98,"GET","//cgi/awstats/",404
64.62.145.98,"GET","//cgi/awstats/",404
64.62.145.98,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi-bin/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi-bin/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi/awstats/",404
64.62.145.98,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
64.62.

64.62.145.98,"GET","/",403
64.62.145.98,"GET","//usage/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//usage/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cgi/awstats/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//usage/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//awstats/perl/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//usage/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","/",403
64.62.145.98,"GET","//.../awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//cp/awstats/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//awstats/perl/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","/",403
64.62.145.98,"GET","//usage/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//usage/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//usage/cgi-bin/awstats.pl?configdir=|%20id%20|",404
64.62.145.98,"GET","//stat-cgi/awstats.pl?configdir=|%20id%2

80.100.183.164,"GET","/scripts/..%255c%255c../winnt/system32/cmd.exe?/c+ver",404
67.89.209.82,"GET","/",403
24.234.222.173,"GET","/",403
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
210.127.248.52,"GET","/sumthin",404
222.166.160.116,"GET","/",403
63.10

210.51.12.238,"GET","//cgi-bin/awstats/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","/",403
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","/",403
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","/",403
210.51.12.238,"GET","//cgi-bin/awstats/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","/",403
210.51.12.238,"GET","/",403
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","/",403
210.51.12.238,"GET","//cgi/awstats.pl?configdir=|%20id%20|",404
210.51.12.238,"GET","//cgi/awstats.pl?co

...

222.95.35.200,"GET","http://ad.trafficmp.com/tmpad/banner/ad/tmp.asp?poID=el0w",404
222.95.35.200,"GET","http://t.trafficmp.com/p.t/i9085/34027962/?http://www.cashtop.net/",404
222.95.35.200,"GET","http://ad.trafficmp.com/tmpad/banner/ad/tmp.asp?poID=el0D",404
222.95.35.200,"GET","http://ad.trafficmp.com/tmpad/banner/ad/tmp.asp?poID=e4V4",404
222.95.32.114,"GET","http://t.trafficmp.com/b.t/i10459/57833442/?Http://www.bbxiong.com",404
222.95.32.114,"GET","http://ad.trafficmp.com/tmpad/banner/ad/tmp.asp?poID=elXb",404
222.95.32.114,"GET","http://ad.trafficmp.com/tmpad/banner/ad/tmp.asp?poID=el0w",404
66.134.74.208,"GET","/",403
222.95.32.114,"GET","http://ad.trafficmp.com/tmpad/banner/ad/tmp.asp?poID=e4V4",404
169.152.251.99,"GET","/",403
222.95.32.114,"GET","http://www.afcyhf.com/image-1505434-10361657",404
222.95.32.114,"GET","http://t.trafficmp.com/b.t/i7007/63619330/?http://www.jebest.com/",404
222.95.32.114,"GET","http://ad.trafficmp.com/tmpad/banner/ad/tmp.asp?poID=elvu",404
24.19.

63.202.179.73,"GET","/msadc/..%255c../..%255c../..%255c/..%c1%1c../..%c1%1c../..%c1%1c../winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET","/scripts/..%c1%1c../winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET","/scripts/..%c0%2f../winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET","/scripts/..%c0%af../winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET","/scripts/..%c1%9c../winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET","/scripts/..%%35%63../winnt/system32/cmd.exe?/c+dir",400
63.202.179.73,"GET","/scripts/..%%35c../winnt/system32/cmd.exe?/c+dir",400
63.202.179.73,"GET","/scripts/..%25%35%63../winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET","/scripts/..%252f../winnt/system32/cmd.exe?/c+dir",404
137.65.139.20,"GET","/",403
63.202.179.73,"GET","/scripts/root.exe?/c+dir",404
63.202.179.73,"GET","/MSADC/root.exe?/c+dir",404
63.202.179.73,"GET","/c/winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET","/d/winnt/system32/cmd.exe?/c+dir",404
63.202.179.73,"GET",

61.178.79.227,"CONNECT","65.54.190.230:25",405
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"CONNECT","66.94.230.39:80",405
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"CONNECT","66.94.230.39:80",405
61.178.79.227,"CONNECT","67.28.114.35:25",405
61.178.79.227,"CONNECT","66.94.230.39:80",405
61.178.79.227,"GET","http://www.yahoo.com/",403
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"CONNECT","67.28.114.35:25",405
61.178.79.227,"GET","http://www.yahoo.com/",403
61.178.79.227,"CONNECT","205.188.158.121:25",405
61.178.79.227,"CONNECT","68.142.202.112:25",405
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"CONNECT","65.54.190.230:25",405
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227,"GET","http://www.ebay.com/",403
61.178.79.227

65.117.45.251,"POST","/_vti_bin/_vti_aut/fp30reg.dll",404
65.117.45.251,"POST","/_vti_bin/_vti_aut/fp30reg.dll",404
222.83.129.198,"GET","/",403
216.70.251.2,"GET","/",403
68.223.144.102,"GET","/",403
221.184.100.13,"GET","/scripts/root.exe?/c+dir",404
221.184.100.13,"GET","/MSADC/root.exe?/c+dir",404
221.184.100.13,"GET","/c/winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/d/winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/scripts/..%255c../winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/_vti_bin/..%255c../..%255c../..%255c../winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/_mem_bin/..%255c../..%255c../..%255c../winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/msadc/..%255c../..%255c../..%255c/..%c1%1c../..%c1%1c../..%c1%1c../winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/scripts/..%c1%1c../winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/scripts/..%c0%2f../winnt/system32/cmd.exe?/c+dir",404
221.184.100.13,"GET","/scrip

220.28.212.176,"GET","/scripts/..%c1%1c../winnt/system32/cmd.exe?/c+dir",404
220.28.212.176,"GET","/scripts/..%c0%2f../winnt/system32/cmd.exe?/c+dir",404
220.28.212.176,"GET","/scripts/..%c0%af../winnt/system32/cmd.exe?/c+dir",404
220.28.212.176,"GET","/scripts/..%c1%9c../winnt/system32/cmd.exe?/c+dir",404
220.28.212.176,"GET","/scripts/..%%35%63../winnt/system32/cmd.exe?/c+dir",400
220.28.212.176,"GET","/scripts/..%%35c../winnt/system32/cmd.exe?/c+dir",400
220.28.212.176,"GET","/scripts/..%25%35%63../winnt/system32/cmd.exe?/c+dir",404
220.28.212.176,"GET","/scripts/..%252f../winnt/system32/cmd.exe?/c+dir",404
143.127.131.4,"GET","/",403
209.137.237.154,"GET","/",403
61.180.25.164,"GET","/",403
81.6.250.202,"GET","/scripts/..%255c%255c../winnt/system32/cmd.exe?/c+ver",404
35.11.228.62,"GET","/",403
216.10.108.138,"GET","/",403
64.85.244.116,"GET","/",403
61.186.47.103,"GET","/",403
210.139.114.144,"GET","/",403
61.30.127.4,"GET","/sumthin",404
61.30.127.4,"GET","/sumthin",404
61.30.127

In [15]:
# extracting only top 10 IP records to other file 
head -n 11 $client_ips > top_ips.csv
cat top_ips.csv

"ip","count"
64.62.145.98,414
210.118.169.20,410
210.116.59.164,179
4.152.207.238,94
210.51.12.238,92
64.122.238.114,91
220.170.88.36,63
222.95.35.200,51
4.152.207.126,46
210.127.248.52,46


In [16]:
# filter the records that includes the above ip addresses. 
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d S:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_pp -f,aq,eok,qui - -cmb,+1,req top_ips.csv S:IP I:count -c IP return_code

"ip","return_code"
2019-12-10 22:46:48 ip-10-10-1-118 ess[4367]: !!!Warning!!! License is going to expire on 2020-01-01!
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",404
"64.62.145.98",403
"64.62.145.98",404
"64.6

"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"210.127.248.52",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.114",404
"64.122.238.1

"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.20",404
"210.118.169.

"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"210.116.59.164",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404
"4.152.207.126",404


In [18]:
# count the occurance counts of return code per each IP addresses
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d S:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_pp -f,aq,eok,qui - -cmb,+1,req top_ips.csv S:IP I:count -o,aq - -c IP return_code | \
aq_cnt -f,aq,eok,qui - -g ip -kX,aq - key return_code | \
aq_ord -f,aq,eok,qui - -sort ip return_code

"ip","return_code","count"
2019-12-10 22:50:13 ip-10-10-1-118 ess[4463]: !!!Warning!!! License is going to expire on 2020-01-01!
"210.116.59.164",404,179
"210.118.169.20",403,23
"210.118.169.20",404,387
"210.127.248.52",404,46
"210.51.12.238",200,23
"210.51.12.238",403,23
"210.51.12.238",404,46
"220.170.88.36",403,63
"222.95.35.200",404,51
"4.152.207.126",404,46
"4.152.207.238",404,94
"64.122.238.114",404,91
"64.62.145.98",200,49
"64.62.145.98",403,46
"64.62.145.98",404,276
"64.62.145.98",500,43


Visualize the result to get a better idea, the graph looks like below. 

<img src='img/status_ip.png'>

It looks like majority of the requests are returned with error code, instead of 200 status code. 404 is the majority of them, which indicates that server is unable to provide the requested resources for whatever reason. 

## Seasonality or Trend in the data

Let's look at the trends in data now. 
How are the seasonality in the data in terms of 
* time of the day
* date of the month

In [25]:
# seasonlity - time of the day
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d S:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_pp -f,aq,eok,qui - -o,aq - | \
aq_cnt -f,aq,eok,qui - -kX,aq - key hour | \
aq_ord -f,aq,eok,qui - -sort hour

"hour","count"
2019-12-10 23:52:07 ip-10-10-1-118 ess[4935]: !!!Warning!!! License is going to expire on 2020-01-01!
0,99
1,78
2,110
3,122
4,99
5,511
6,97
7,64
8,141
9,89
10,114
11,244
12,125
13,60
14,72
15,122
16,492
17,57
18,61
19,60
20,72
21,48
22,51
23,160


Visualizing the result, we get 



In [26]:
# Seasonality - date of the month
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d S:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' I:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_pp -f,aq,eok,qui - -o,aq - | \
aq_cnt -f,aq,eok,qui - -kX,aq - key date | \
aq_ord -f,aq,eok,qui - -sort date

"date","count"
2019-12-10 23:56:42 ip-10-10-1-118 ess[5023]: !!!Warning!!! License is going to expire on 2020-01-01!
1,277
2,66
3,83
4,119
5,118
6,457
7,317
8,183
9,520
10,50
11,88
12,55
13,202
14,82
15,42
16,51
17,21
22,22
23,43
24,72
25,64
26,55
27,37
28,40
30,70
31,14


The graph looks like below.

We can plot the two information in one graph as a heatmap. In order to do this, we need a pivot table with hours of the day as column, and date of the month as index. Let's take a look how we can acheive this.

In [36]:
# pivot table
ess stream weblogs "*" "*" | aq_pp -f,+1,div,eok,qui - \
-d S:ip sep:' ' S:hypen sep:' ' S:user_id sep:' [' I:date sep:'/' S:month sep:'/' I:year sep:':' S:hour sep:':' I:minute sep:':' I:second sep:' ' S:time_dif sep:'] "' S:request_method sep:' ' S:requested_resource sep:' ' S:protocol sep:'" ' I:return_code sep:' ' I:obj_size sep:' ' S:referrer sep:' ' S:user_agent -o,aq - | \
aq_pp -f,aq,eok,qui - -o,aq - | \
aq_cnt -f,aq,eok,qui - -g date -kX,aq - key hour | \
aq_rst -f,aq,eok - -key date -lab hour -val count

2019-12-11 00:12:33 ip-10-10-1-118 ess[5500]: !!!Warning!!! License is going to expire on 2020-01-01!
"date","00","01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23"
1,1,1,0,1,24,0,6,18,62,19,16,38,44,2,1,1,1,0,24,2,0,0,0,16
2,0,1,0,0,0,1,0,2,0,1,1,1,16,1,1,0,3,1,2,0,1,17,1,16
3,0,1,1,24,0,0,1,0,1,0,0,1,0,0,0,26,16,2,2,1,1,1,2,3
4,0,1,1,2,3,16,0,3,1,0,1,2,1,2,3,46,0,0,1,1,16,3,0,16
5,1,3,2,2,4,1,10,27,2,0,1,1,2,3,0,16,0,1,0,17,0,1,1,23
6,2,1,1,1,0,1,2,3,1,3,2,0,1,16,1,0,415,2,0,0,1,2,2,0
7,23,1,1,25,1,2,0,2,16,42,19,113,8,3,2,1,3,15,11,3,3,4,2,17
8,3,6,4,1,4,2,20,5,10,5,7,45,5,7,5,7,19,4,4,6,6,5,1,2
9,3,7,4,6,20,411,1,0,0,1,1,2,26,18,16,0,0,0,1,0,0,2,0,1
10,1,2,0,1,1,4,1,1,22,1,1,1,2,1,1,2,0,0,5,1,1,0,1,0
11,1,1,1,0,0,0,19,0,0,1,23,19,0,1,0,0,2,0,1,2,16,0,1,0
12,0,1,1,3,1,2,35,1,1,1,1,1,0,0,2,2,0,0,0,0,1,0,1,1
13,26,46,47,52,10,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,16
14,17,2,2,0,1,17,0,0,0,1,16,0,0,0,0,0,21,1,0,1,0,2,0,1
15,